# Analyzing Samsung Health Data

Concepts and methodologies for parsing, cleaning, and visualizing Samsung Health exports are taken from
[Analyzing Samsung Health Data](https://www.kaggle.com/code/kmader/analyzing-samsung-health-data/notebook#Samsung-Data) notebook by K. Scott Mader.

I have adapt that work to align with the latest update of the Samsung Health app, version 6.30.5.105, released on September 23, 2025.

## Extracting Data from Samsung Health

Exporting Personal Data (CSV/JSON) directly from the app:
1. Open The App in your phone.
2. Access the Menu tapping the three vertical dots (or a similar menu icon) usually located in the top-right corner.
3. Select Settings from the menu.
4. Look for and select the option `Download personal data` (or `Export data`).
5. The application will process and download the data into a compressed file on your device.
   You can usually find the exported files in the device's Download folder.

### File format
The exported data is typically provided in CSV (.csv) and JSON (.json) formats.

Next, we'll use Python code snippets to open and analyze the health data stored in CSV files.

## Setup and Function Definition

The first row always contains a metadata line.
The actual header appears in the next row.

Each data row may start with an empty leading column (a leading comma),
then pandas will assume that first column is the DataFrame index,
shifting all columns left so values end up under misaligned headers.

Curiously, that happens even if the header has a name for it.

In [1]:
# Use the Matplotlib inline magic command (if not already set)
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

samsung_dump_dir = 'samsunghealth_data'

# Define an anonymous function that reads CSV files
# using the pandas library with these specific arguments:
#   `skiprows=1`: ignore the first metadata line.
#   `index_col=False`: ensure the first column is treated as a regular data column.
sam_readcsv = lambda x: pd.read_csv(x, skiprows=1, index_col=False)

## Plot the heart rate column against the time

In [ ]:
heart_rate_tracker = os.path.join(samsung_dump_dir, 'com.samsung.shealth.tracker.heart_rate.20251018112612.csv')
heart_rate_df = sam_readcsv(heart_rate_tracker)

# Select the relevant columns (heart rate and datetime)
hr_column_name = 'com.samsung.health.heart_rate.heart_rate'
time_column_name = 'com.samsung.health.heart_rate.end_time'

# Convert the time column to datetime objects
heart_rate_df[time_column_name] = pd.to_datetime(heart_rate_df[time_column_name])

# Set the datetime column as the DataFrame index
heart_rate_df = heart_rate_df.set_index(time_column_name)

# Create the figure and axes
plt.figure(figsize=(12, 6))

# Use the pandas plot method (which uses Matplotlib internally)
# Alternatively, you could use hr_data.plot(ax=ax)
heart_rate_df[hr_column_name].plot(title='Heart Rate Over Time', grid=True)

# Label the axes
plt.xlabel("Time")
plt.ylabel("Heart Rate (BPM)")

# Display the plot
plt.show()

## Plot systolic (SYS) and diastolic (DIA) blood pressure against time

In [ ]:
blood_pressure_tracker = os.path.join(samsung_dump_dir, 'com.samsung.shealth.blood_pressure.20251018112612.csv')
blood_pressure_df = sam_readcsv(blood_pressure_tracker)

sys_column_name = 'com.samsung.health.blood_pressure.systolic'
dia_column_name = 'com.samsung.health.blood_pressure.diastolic'
time_column_name = 'com.samsung.health.blood_pressure.update_time'

# Convert the time column to datetime objects
blood_pressure_df[time_column_name] = pd.to_datetime(blood_pressure_df[time_column_name])

# Set the datetime column as the DataFrame index
blood_pressure_df = blood_pressure_df.set_index(time_column_name)

# Select only the blood pressure columns for plotting
bp_data = blood_pressure_df[[sys_column_name, dia_column_name]].copy()

# Create the figure and axes
plt.figure(figsize=(14, 7))

# Plot both columns. Pandas uses the column names as the legend labels.
ax = bp_data.plot(
    title='Blood Pressure Over Time (Systolic and Diastolic)',
    grid=True,
    figsize=(14, 7) # Re-specifying size just in case
)

# Rename the legend labels for clarity
ax.legend(['Systolic (SYS)', 'Diastolic (DIA)'])

# Label the axes
plt.xlabel("Time")
plt.ylabel("Blood Pressure (mmHg)")

plt.tight_layout() # Adjusts plot to prevent labels from overlapping
plt.savefig('blood_pressure_plot.png')
plt.close()

## Process Samsung CSV files

In [4]:
print('Dump dir:', samsung_dump_dir)

samsung_csv_paths = glob(os.path.join(samsung_dump_dir, '*.csv'))
print(len(samsung_csv_paths), 'csvs found')
samsung_json_paths = glob(os.path.join(samsung_dump_dir, 'jsons', '*'))
print(len(samsung_json_paths), 'jsons found')


Dump dir: samsunghealth_data
52 csvs found
30 jsons found


In [5]:
# 1. Import the display function from IPython.display
# ---------------------------------------------------
# This function is essential in Jupyter Notebooks for rendering objects
# (like pandas DataFrames) in a visually appealing HTML format,
# rather than just raw text output.
from IPython.display import display

# 2. Loading All CSV Files into a Dictionary
# ------------------------------------------
# Loads every specified CSV file into a single dictionary called `all_csv_df` (dictionary comprehension).
# The dictionary keys consist of the filename of the single CSV path removing the common prefix (`com.samsung.`).
all_csv_df = {os.path.basename(j).replace('com.samsung.', ''): sam_readcsv(j) for j in samsung_csv_paths}

# 3. Iteration and Inspection
# ---------------------------
# Start a loop to iterate through the key-value pairs of the `all_csv_df` dictionary:
#   k = The clean filename/key (e.g., `heart_rate.csv`).
#   v = The pandas DataFrame (the data itself).
for k, v in all_csv_df.items():
    # Print the name of the dataset and the total number of records it contains.
    #   v.shape[0] returns the number of rows in the DataFrame (the number of readings).
    print(k, 'readings:', v.shape[0])

    # Display a small, random sample of the data from the current DataFrame.
    # Select 2 random rows if the DataFrame has at least 3 rows, or just 1 row if it has 2 or fewer rows.
    #display(v.sample(2 if v.shape[0]>2 else 1))

    # Display the column names as the first row of a nicely formatted table (like a DataFrame header).
    # First of all, create a single-row DataFrame from the column names.
    # The .to_frame().T method is a neat trick:
    # 1. to_frame() converts the Index (names) into a one-column DataFrame.
    # 2. .T (transpose) flips it, making the names the header row.
    columns_df = v.columns.to_frame().T
    # Then display the temporary DataFrame (which looks like a header row).
    display(columns_df)

health.advanced_glycation_endproduct.20251018112612.csv readings: 104


,create_sh_ver,measurement_result,percent,modify_sh_ver,update_time,create_time,score,deviceuuid,level_boundary,pkg_name,datauuid,day_time
0,create_sh_ver,measurement_result,percent,modify_sh_ver,update_time,create_time,score,deviceuuid,level_boundary,pkg_name,datauuid,day_time


health.advanced_glycation_endproduct.raw.20251018112612.csv readings: 104


,create_sh_ver,start_time,binning_data,modify_sh_ver,update_time,create_time,version,time_offset,deviceuuid,pkg_name,end_time,datauuid
0,create_sh_ver,start_time,binning_data,modify_sh_ver,update_time,create_time,version,time_offset,deviceuuid,pkg_name,end_time,datauuid


health.device_profile.20251018112612.csv readings: 7


...

,create_sh_ver,step_count,binning_data,active_time,recommendation,modify_sh_ver,run_step_count,update_time,source_package_name,create_time,...,speed,distance,calorie,walk_step_count,deviceuuid,pkg_name,healthy_step,achievement,datauuid,day_time
0,create_sh_ver,step_count,binning_data,active_time,recommendation,modify_sh_ver,run_step_count,update_time,source_package_name,create_time,...,speed,distance,calorie,walk_step_count,deviceuuid,pkg_name,healthy_step,achievement,datauuid,day_time


shealth.tracker.pedometer_step_count.20251018112612.csv readings: 7335


,duration,version_code,run_step,walk_step,com.samsung.health.step_count.start_time,com.samsung.health.step_count.sample_position_type,com.samsung.health.step_count.custom,com.samsung.health.step_count.update_time,com.samsung.health.step_count.create_time,com.samsung.health.step_count.count,com.samsung.health.step_count.speed,com.samsung.health.step_count.distance,com.samsung.health.step_count.calorie,com.samsung.health.step_count.time_offset,com.samsung.health.step_count.deviceuuid,com.samsung.health.step_count.pkg_name,com.samsung.health.step_count.end_time,com.samsung.health.step_count.datauuid
0,duration,version_code,run_step,walk_step,com.samsung.health.step_count.start_time,com.samsung.health.step_count.sample_position_...,com.samsung.health.step_count.custom,com.samsung.health.step_count.update_time,com.samsung.health.step_count.create_time,com.samsung.health.step_count.count,com.samsung.health.step_count.speed,com.samsung.health.step_count.distance,com.samsung.health.step_count.calorie,com.samsung.health.step_count.time_offset,com.samsung.health.step_count.deviceuuid,com.samsung.health.step_count.pkg_name,com.samsung.health.step_count.end_time,com.samsung.health.step_count.datauuid


shealth.vitality_score.20251018112612.csv readings: 55


,activity_level,activity_score,create_sh_ver,active_time_scale,activity_balance_short_term_activity,activity_balance_long_term_activity,shr_balance_scale,mvpa_time_optimal_range_max,mvpa_time_optimal_range_min,max_hr,...,active_time_range_max,active_time_range_min,shrv_baseline_max,shrv_baseline_min,datauuid,prev_shr_avg,day_time,shrv_penalty,activity_balance_optimal_range_max,activity_balance_optimal_range_min
0,activity_level,activity_score,create_sh_ver,active_time_scale,activity_balance_short_term_activity,activity_balance_long_term_activity,shr_balance_scale,mvpa_time_optimal_range_max,mvpa_time_optimal_range_min,max_hr,...,active_time_range_max,active_time_range_min,shrv_baseline_max,shrv_baseline_min,datauuid,prev_shr_avg,day_time,shrv_penalty,activity_balance_optimal_range_max,activity_balance_optimal_range_min
